In [3]:
from elasticsearch import Elasticsearch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from langchain.schema import Document

In [4]:
es_client = Elasticsearch(
    hosts=["http://loacalhost:9200"],
    http_auth=('elastic', '**********'),
)

/tmp/ipykernel_70071/2547814963.py:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es_client = Elasticsearch(


In [5]:
es_client.info()

ObjectApiResponse({'name': 'node-1', 'cluster_name': 'ksy_cl', 'cluster_uuid': 'TSNnkHhQQGSIHlLRuOCB9w', 'version': {'number': '8.12.2', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '48a287ab9497e852de30327444b0809e55d46466', 'build_date': '2024-02-19T10:04:32.774273190Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
docs = [
    Document(
        page_content="많은 과학자들이 공룡을 되살려내고 대혼란을 일으킵니다",
        metadata={"id": 1,"name":"kim"},
    ),
    Document(
        page_content="레오 디카프리오는 꿈속에서 꿈속에서 길을 잃습니다...",
        metadata={"id": 2,"name":"lee"},
    ),
    Document(
        page_content="심리학자/탐정이 꿈속에서 꿈속에서 일련의 꿈속에서 길을 잃고 인셉은 이 아이디어를 재사용합니다",
        metadata={"id": 3,"name":"choi"},
    ),
    Document(
        page_content="보통 크기의 여자들은 아주 건강하고 어떤 남자들은 그들을 따라 합니다",
        metadata={"id": 4,"name":"park"},
    ),
    Document(
        page_content="장난감은 살아나고 그렇게 하면서 즐거운 시간을 보냅니다",
        metadata={"id": 5,"name":"seo"},
    ),
    Document(
        page_content="세명의 남자가 존 안으로 들어가고, 세명의 남자가 존 밖으로 나갑니다",
        metadata={"id": 6,"name":"bae"},
    ),
]


model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

In [9]:
db = ElasticsearchStore.from_documents(
    docs,
    hf,
    es_connection=es_client,
    index_name="test1",
    distance_strategy="COSINE"
    #distance_strategy="EUCLIDEAN_DISTANCE"
    #distance_strategy="DOT_PRODUCT"
)

In [10]:
query = "장난감 나오는 영화가 뭐야?"
results = db.similarity_search(query,k=3)
print(results)

[Document(page_content='장난감은 살아나고 그렇게 하면서 즐거운 시간을 보냅니다', metadata={'id': 5, 'name': 'seo'}), Document(page_content='심리학자/탐정이 꿈속에서 꿈속에서 일련의 꿈속에서 길을 잃고 인셉은 이 아이디어를 재사용합니다', metadata={'id': 3, 'name': 'choi'}), Document(page_content='레오 디카프리오는 꿈속에서 꿈속에서 길을 잃습니다...', metadata={'id': 2, 'name': 'lee'})]


In [11]:
docs = db.similarity_search(query)
print(docs[0].metadata)

{'id': 5, 'name': 'seo'}


In [29]:
docs = db.similarity_search(
    query, filter=[{"term": {"metadata.name.keyword": "park"}}]
)
print(docs[0].metadata)

{'id': 4, 'name': 'park'}


In [16]:
db = ElasticsearchStore.from_documents(
    docs,
    hf,
    es_connection=es_client,
    index_name="test2",
    distance_strategy="COSINE",
    #distance_strategy="EUCLIDEAN_DISTANCE"
    #distance_strategy="DOT_PRODUCT",
    strategy=ElasticsearchStore.ApproxRetrievalStrategy()
)

In [17]:
query = "장난감 나오는 영화가 뭐야?"
results = db.similarity_search(query,k=3)
print(results)

[Document(page_content='장난감은 살아나고 그렇게 하면서 즐거운 시간을 보냅니다', metadata={'id': 5, 'name': 'seo'}), Document(page_content='심리학자/탐정이 꿈속에서 꿈속에서 일련의 꿈속에서 길을 잃고 인셉은 이 아이디어를 재사용합니다', metadata={'id': 3, 'name': 'choi'}), Document(page_content='레오 디카프리오는 꿈속에서 꿈속에서 길을 잃습니다...', metadata={'id': 2, 'name': 'lee'})]
